# test MySQL and Postgres dataset connections

first, we've deployed the database instances from the ../databases/docker-compose.yaml file.

here, we've deployed them to my local server, and will connect remotely and execute a query.

In [1]:
import sys
sys.path.append("../src")
from text2sql import hello
print(hello.message)

hello, world!


In [2]:
import os

from dotenv import load_dotenv

from text2sql.data import MysqlDataset, PostgresDataset

load_dotenv()

True

## test MySQL

the dummy databases are "washswat_en" and "washswat_ko"

In [3]:
mysql_dataset = MysqlDataset(
    "jennifer.dereks.house",
    3306,
    "root",
    os.getenv("MYSQL_ROOT_PASSWORD"),
)

In [4]:
# for mysql, this gets all databases immediately
print(mysql_dataset.databases)

['information_schema', 'mysql', 'performance_schema', 'sys', 'washswat_en', 'washswat_ko']


In [5]:
# get schema description
schema_text = mysql_dataset.describe_database_schema("washswat_en", mode="basic_types_relations")
print(schema_text)

table 'item' with columns: item_id (int) , order_id (varchar(80)) , barcode (varchar(100)) , name (varchar(80)) , status (varchar(80)) , price (bigint) , option (varchar(80)) , shop (varchar(80))
table 'item_price' with columns: item_price_id (bigint) , name (varchar(80)) , category (varchar(80)) , price_basic (bigint) , price_premium (bigint) , description (varchar(1000))
table 'order_list' with columns: order_id (varchar(80)) , user_id (bigint) , created_at (timestamp(6)) , address (varchar(256)) , road_address (varchar(256)) , post_code (varchar(256)) , total_price (bigint) , status (varchar(80)) , expected_pickup_at (timestamp(6)) , expected_delivery_at (timestamp(6)) , pickup_complete_at (timestamp(6)) , delivery_complete_at (timestamp(6))

Relations:
item.order_id -> order_list.order_id


In [6]:
# run a query on a database
mysql_dataset.query_database("washswat_ko", "SELECT DISTINCT status FROM item;")

[{'status': 'compensationOver'},
 {'status': 'complete'},
 {'status': 'inOutboundStorage'},
 {'status': 'inspectionComplete'},
 {'status': 'issueItem'},
 {'status': 'mptLoaded'},
 {'status': 'mptScan'},
 {'status': 'needConfirm'},
 {'status': 'ospartner'},
 {'status': 'ospartnerDeparture'},
 {'status': 'ospartnerReturn'},
 {'status': 'outboundStorageDeparture'},
 {'status': 'packing'},
 {'status': 'postProcessReLaundry'},
 {'status': 'prepareOutboundStorageDeparture'},
 {'status': 'pricing'},
 {'status': 'recyclingComplete'},
 {'status': 'reLaundry'},
 {'status': 'repairNeeded'},
 {'status': 'sorting'},
 {'status': 'tagging'},
 {'status': 'washingComplete'}]

## test Postgres

should only have "genapostgre"

In [7]:
postgres_dataset = PostgresDataset(
    "jennifer.dereks.house",
    5432,
    "genapostgre",
    os.getenv("POSTGRES_PASSWORD"),
)

In [8]:
# get schema description
schema_text = postgres_dataset.describe_database_schema("genapostgre", mode="basic_types_relations")
print("\n".join(schema_text.split("\n")[:3]))

table 'email' with columns: id (uuid) , created_at (timestamp without time zone) , updated_at (timestamp without time zone) , deleted_at (timestamp without time zone) , metadata (jsonb) , to (ARRAY) , cc (ARRAY) , bcc (ARRAY) , subject (character varying) , language (character varying) , body (text) , is_opened (boolean) , first_opened_at (timestamp without time zone) , last_opened_at (timestamp without time zone) , entity_type (character varying) , entity_id (uuid) , tenant_id (uuid) , creator_id (uuid) , modifier_id (uuid)
table 'api_key' with columns: id (uuid) , created_at (timestamp without time zone) , updated_at (timestamp without time zone) , deleted_at (timestamp without time zone) , name (character varying) , value (character varying) , tenant_id (uuid) , user_id (uuid) , metadata (jsonb)
table 'business_registration' with columns: id (uuid) , created_at (timestamp without time zone) , updated_at (timestamp without time zone) , deleted_at (timestamp without time zone) , regis

In [9]:
# run a query on a database
postgres_dataset.query_database("genapostgre", "SELECT COUNT(DISTINCT bank) FROM bank_account;")

[{'count': 10}]